In [1]:
import pandas as pd
import pymysql.cursors
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np

In [2]:
user='kazi_prod'
password='kazi_prod'
host='kazi-prod.c8g7cvnd0wr8.eu-west-2.rds.amazonaws.com'
database='kazi'
    
connc = pymysql.connect(user=user, password=password, host=host, database=database)
cursor = connc.cursor()

In [3]:
df = pd.read_sql('SELECT * FROM task_call_details', con=connc)

In [4]:
print(df.shape)
df.head()

(77993, 13)


,id,angaza_id,account_number,call_duration,task_solved_at,followup_date,intent_id,feedback_id,other_reason,imei,contacted_number,created_at,updated_at
0,3499,US014637,3179876,55,2018-10-11 05:10:58,2018-10-16,2,999,Language problems,351701094167580,None,2018-10-11 05:11:01,2018-10-11 05:11:01
1,3500,US014637,3866878,117,2018-10-11 05:16:40,2018-10-15,2,4,None,351701094167580,None,2018-10-11 05:16:42,2018-10-11 05:16:42
2,3501,US011921,15270002,62,2018-10-11 05:20:50,2018-10-26,2,4,None,352452090251304,None,2018-10-11 05:20:52,2018-10-11 05:20:52
3,3502,US014637,11620002,118,2018-10-11 05:23:58,2018-10-12,2,4,None,351701094167580,None,2018-10-11 05:24:02,2018-10-11 05:24:02
4,3503,US014637,13058706,21,2018-10-11 05:26:08,2018-10-13,2,5,None,351701094167580,None,2018-10-11 05:26:10,2018-10-11 05:26:10


In [5]:
df.dtypes

id                           int64
angaza_id                   object
account_number               int64
call_duration                int64
task_solved_at      datetime64[ns]
followup_date               object
intent_id                    int64
feedback_id                  int64
other_reason                object
imei                        object
contacted_number            object
created_at          datetime64[ns]
updated_at          datetime64[ns]
dtype: object

In [6]:
from datetime import datetime

In [7]:
index_list = []
for index, row in df.iterrows():
    created_at = row['created_at']
    task_solved_at = row['task_solved_at']
    if (task_solved_at.hour == 0 and task_solved_at.minute == 0 and task_solved_at.second == 0):
        index_list.append(index)

    if (created_at.hour == 0 and created_at.minute == 0 and created_at.second == 0):
        index_list.append(index)
    #print(row['created_at'], row['task_solved_at'])
#print(index_list)

In [8]:
print('before drop', df.shape)

df.drop(df.index[index_list], inplace=True)

print('after drop', df.shape)


before drop (77993, 13)
after drop (75431, 13)


In [9]:
df['sync_delay'] = df.created_at - df.task_solved_at
df['sync_delay'] = df['sync_delay'] / np.timedelta64(1, 's')

In [10]:
df.head()

,id,angaza_id,account_number,call_duration,task_solved_at,followup_date,intent_id,feedback_id,other_reason,imei,contacted_number,created_at,updated_at,sync_delay
0,3499,US014637,3179876,55,2018-10-11 05:10:58,2018-10-16,2,999,Language problems,351701094167580,None,2018-10-11 05:11:01,2018-10-11 05:11:01,3.0
1,3500,US014637,3866878,117,2018-10-11 05:16:40,2018-10-15,2,4,None,351701094167580,None,2018-10-11 05:16:42,2018-10-11 05:16:42,2.0
2,3501,US011921,15270002,62,2018-10-11 05:20:50,2018-10-26,2,4,None,352452090251304,None,2018-10-11 05:20:52,2018-10-11 05:20:52,2.0
3,3502,US014637,11620002,118,2018-10-11 05:23:58,2018-10-12,2,4,None,351701094167580,None,2018-10-11 05:24:02,2018-10-11 05:24:02,4.0
4,3503,US014637,13058706,21,2018-10-11 05:26:08,2018-10-13,2,5,None,351701094167580,None,2018-10-11 05:26:10,2018-10-11 05:26:10,2.0


In [11]:
minimum = df.sync_delay.min()
#minimum = minimum.total_seconds()
maximum = df.sync_delay.max()
#maximum = maximum.total_seconds()

In [12]:
bins=[minimum, -15, 15, maximum]
labels = ['offline_less','online','offline_more']

In [13]:
bins

[-20131197.0, -15, 15, 92633847.0]

In [14]:
df['sync_state'] = pd.cut(df['sync_delay'], bins, labels = labels )

In [15]:
df

,id,angaza_id,account_number,call_duration,task_solved_at,followup_date,intent_id,feedback_id,other_reason,imei,contacted_number,created_at,updated_at,sync_delay,sync_state
0,3499,US014637,3179876,55,2018-10-11 05:10:58,2018-10-16,2,999,Language problems,351701094167580,None,2018-10-11 05:11:01,2018-10-11 05:11:01,3.0,online
1,3500,US014637,3866878,117,2018-10-11 05:16:40,2018-10-15,2,4,None,351701094167580,None,2018-10-11 05:16:42,2018-10-11 05:16:42,2.0,online
2,3501,US011921,15270002,62,2018-10-11 05:20:50,2018-10-26,2,4,None,352452090251304,None,2018-10-11 05:20:52,2018-10-11 05:20:52,2.0,online
3,3502,US014637,11620002,118,2018-10-11 05:23:58,2018-10-12,2,4,None,351701094167580,None,2018-10-11 05:24:02,2018-10-11 05:24:02,4.0,online
4,3503,US014637,13058706,21,2018-10-11 05:26:08,2018-10-13,2,5,None,351701094167580,None,2018-10-11 05:26:10,2018-10-11 05:26:10,2.0,online
5,3504,US014637,13205901,69,2018-10-11 05:29:10,2018-10-16,2,4,None,351701094167580,None,2018-10-11 05:29:14,2018-10-11 05:29:14,4.0,online
6,3505,US014637,22207609,134,2018-10-11 05:34:19,2018-10-14,2,3,None,351701094167580,None,2018-10-11 05:34:22,2018-10-11 05:34:22,3.0,online
7,3506,US014637,24682502,37,2018-10-11 05:37:20,2018-10-13,2,4,None,351701094167580,None,2018-10-11 05:37:22,2018-10-11 05:37:22,2.0,online
8,3507,US014637,23895808,50,2018-10-11 05:39:20,2018-10-13,2,4,None,351701094167580,None,2018-10-11 05:39:22,2018-10-11 05:39:22,2.0,online
9,3508,US014637,32479206,56,2018-10-11 05:42:16,2018-10-13,2,5,None,351701094167580,None,2018-10-11 05:42:19,2018-10-11 05:42:19,3.0,online


In [16]:
online_count = sum(df.sync_state == 'online')
offline_count = sum(df.sync_state != 'online')
percentage = online_count/(online_count + offline_count)*100
print(percentage)

59.04203841921757
